Laces Out!
Or is Seattle Seahawks Special Team good enough?

# I. Points
Points earned by Special Teams over NFL (field-goals and extra-points) 

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import cbook

import warnings
warnings.filterwarnings("ignore")

imageFile0 = cbook.get_sample_data("/kaggle/input/nfl2022/nfl_st_points3.png")
image0 = plt.imread(imageFile0)

plt.rcParams["figure.figsize"] = (5,10)
plt.axis('off')
plt.title('Score points by Special Teams')
plt.imshow(image0)

It's look like SEA Special Team has made ~3% of total Special Teams points over NFL (middle-level)

# II. Probability of field-goal

There is only one way to score by kick - make accurate long snap and make accurate kick

In [ ]:
imageFile1 = cbook.get_sample_data("/kaggle/input/nfl2022/nfl_LS_and_K3.png")
image1 = plt.imread(imageFile1)

plt.rcParams["figure.figsize"] = (10,15)
plt.axis('off')
plt.title('Field-goal scores funnel')
plt.imshow(image1)

'*' - goal or no goal

Ok, the long snap technique the same for punts and field-goals, so i've calculated long snap accuracy with help of Punt data (Punts ~ 50% of LS plays)

for season 2019:
* specialTeamsPlayType             count
* Extra Point                   1117
* Field Goal                     894
* Punt                          2015

So we can calculate probability of field-goal.. Based on 2020year-data goal probability is 0.96 times 0.94 for SEA (0.90)

# III. Blocks

Even if you are good enough in snap and kick, your opponent can make a block (and you will not score)

In [ ]:
imageFile3 = cbook.get_sample_data('/kaggle/input/nfl2022/nfl_blocks.PNG')
image3 = plt.imread(imageFile3)

plt.rcParams["figure.figsize"] = (10,5)
plt.axis('off')
plt.title('Blockes on field-goals')
plt.imshow(image3)

SEA team allowed 1 block and made 1 block in season 2018. Also SEA team made 3 blocks in season 2019.

# IV. Long Snappers

In [ ]:
tracking2018 = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/tracking2018.csv')
tracking2019 = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/tracking2019.csv')
tracking2020 = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/tracking2020.csv')
play = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/plays.csv')
game = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/games.csv')
# combine tracking data
tracking_all0 = pd.concat([
    tracking2018[(tracking2018.position=='LS') & (tracking2018.event =='ball_snap')],
    tracking2019[(tracking2019.position=='LS') & (tracking2019.event =='ball_snap')],
    tracking2020[(tracking2020.position=='LS') & (tracking2020.event =='ball_snap')]],
    axis=0,ignore_index=True)
tracking_all1 = pd.concat([
    tracking2018[(tracking2018.position=='K') & (tracking2018.event =='ball_snap')],
    tracking2019[(tracking2019.position=='K') & (tracking2019.event =='ball_snap')],
    tracking2020[(tracking2020.position=='K') & (tracking2020.event =='ball_snap')]],
    axis=0,ignore_index=True)
tracking_all2 = pd.concat([
    tracking2018[(tracking2018.position=='K') & (tracking2018.event =='field_goal_attempt')],
    tracking2019[(tracking2019.position=='K') & (tracking2018.event =='field_goal_attempt')],
    tracking2020[(tracking2020.position=='K') & (tracking2018.event =='field_goal_attempt')]],
    axis=0,ignore_index=True)

play_fg_ls = pd.merge(play[(play.specialTeamsPlayType=='Field Goal') | (play.specialTeamsPlayType=='Extra Point')],
                   tracking_all0, how='inner',on=["gameId","playId"])

fg_result=[]
for i in range(len(play_fg_ls)):
    if play_fg_ls['specialTeamsResult'][i]=='Kick Attempt Good':
        fg_result.append(0)
    else:
        fg_result.append(1)
play_fg_ls = pd.concat([play_fg_ls,pd.DataFrame(fg_result, columns=['fg_no_good'])], axis=1)  

play_fg_ls = play_fg_ls[['x','y','o' ,'playDirection','fg_no_good']]

for i in range(len(play_fg_ls)):
    if play_fg_ls['playDirection'][i] == 'right':
        play_fg_ls['x'][i] = 120 - play_fg_ls['x'][i]
        play_fg_ls['y'][i] = 53.3 - play_fg_ls['y'][i]
        if (play_fg_ls['o'][i] - 180) <0:
            play_fg_ls['o'][i] = play_fg_ls['o'][i] + 180
        else:
            play_fg_ls['o'][i]=play_fg_ls['o'][i] - 180
#
import math
distance=[]
for i in range(len(play_fg_ls)):
    distance.append(math.sqrt(play_fg_ls['x'][i]**2 + (play_fg_ls['y'][i]-53.3/2)**2))
    
play_fg_ls = pd.concat([play_fg_ls, pd.DataFrame(distance, columns=['distance'])], axis=1)

import seaborn as sns
ax = sns.displot(data=play_fg_ls[['o','fg_no_good']],x="o", binwidth=10, 
                 hue="fg_no_good", multiple="stack")
ax.set(title='LS helmet orientation at moment of snap')

Most of Long Snappers see straight to goal post at moment of snap (270 degrees)

# V. Kickers

And most Kickers see and move in same direction at that moment.

In [ ]:
# Kicker at the moment of Snap
play_fg_k1 = pd.merge(play[(play.specialTeamsPlayType=='Field Goal') | (play.specialTeamsPlayType=='Extra Point')],
                   tracking_all1, how='inner',on=["gameId","playId"])

fg_result=[]
for i in range(len(play_fg_k1)):
    if play_fg_k1['specialTeamsResult'][i]=='Kick Attempt Good':
        fg_result.append(0)
    else:
        fg_result.append(1)
play_fg_k1 = pd.concat([play_fg_k1,pd.DataFrame(fg_result, columns=['fg_no_good'])], axis=1)   

play_fg_k1 = play_fg_k1[['x','y','s','a','o','dir','playDirection','fg_no_good']]
# angle between velocity vector and orientation vector
diff_dir_o = []

for i in range(len(play_fg_k1)):
    if play_fg_k1['o'][i] - play_fg_k1['dir'][i] <-180:
        diff_dir_o.append(play_fg_k1['o'][i] - play_fg_k1['dir'][i] + 360)
    elif play_fg_k1['o'][i] - play_fg_k1['dir'][i] >180:
        diff_dir_o.append(play_fg_k1['o'][i] - play_fg_k1['dir'][i] - 360)
    else:
        diff_dir_o.append(play_fg_k1['o'][i] - play_fg_k1['dir'][i])
    if play_fg_k1['playDirection'][i] == 'right':
        play_fg_k1['x'][i]=120-play_fg_k1['x'][i]
        play_fg_k1['y'][i]=53.3-play_fg_k1['y'][i]
        if (play_fg_k1['o'][i]-180)<0:
            play_fg_k1['o'][i]=play_fg_k1['o'][i]+180
        else:
            play_fg_k1['o'][i]=play_fg_k1['o'][i]-180
        if (play_fg_k1['dir'][i]-180)<0:
            play_fg_k1['dir'][i]=play_fg_k1['dir'][i]+180
        else:
            play_fg_k1['dir'][i]=play_fg_k1['dir'][i]-180
#
    
play_fg_k1 = pd.concat([play_fg_k1, pd.DataFrame(diff_dir_o, columns=['diff_dir_o'])], axis=1)

ax = sns.displot(data=play_fg_k1[['diff_dir_o','fg_no_good']],x="diff_dir_o", binwidth=10, 
            hue="fg_no_good", multiple="stack")
ax.set(title='Angle between orientation and velocity-vector of Kicker')

But some of Kickers look at opposite direction!

Distance to goal-post is the main factor of misses (see Fig. below).

# VI. Long distance field-goals

In [ ]:
# Kickers at the moment of field -goal attempt

play_fg_k2 = pd.merge(play[(play.specialTeamsPlayType=='Field Goal')],
                       tracking2018[(tracking2018.position=='K') & (tracking2018.event =='field_goal_attempt')], 
                       how='inner',on=["gameId","playId"])

fg_result=[]
good = []
attempts = []

for i in range(len(play_fg_k2)):
    if play_fg_k2['specialTeamsResult'][i]=='Kick Attempt Good':
        fg_result.append(0)
        good.append(1)
        attempts.append(1)
    else:
        fg_result.append(1)
        good.append(0)
        attempts.append(1)
play_fg_k2 = pd.concat([play_fg_k2,
                         pd.DataFrame(fg_result, columns=['fg_no_good']),
                         pd.DataFrame(good, columns =['good_kick']),
                         pd.DataFrame(attempts, columns = ['attempt'])], axis=1) 

play_fg_k2 = play_fg_k2[['x','y','possessionTeam','displayName','playDirection',
                         'fg_no_good','good_kick','attempt']]

for i in range(len(play_fg_k2)):
    if play_fg_k2['playDirection'][i]=='right':
        play_fg_k2['x'][i] = 120 - play_fg_k2['x'][i]
        play_fg_k2['y'][i] = 53.3 - play_fg_k2['y'][i]
    
ax =sns.displot(data=play_fg_k2[['x','fg_no_good']],x="x", binwidth=2, 
            hue="fg_no_good", multiple="stack")
ax.set(title="Field-goal results depend on X-axis (2018)")

It is look like very hard to score, when distance to goal post is more then 51 yards (35 yard line marker, x>=45).
So we can calculate amount of good long distance kicks for kickers and accuacy fo that kicks.

In [ ]:
play_fg_k2 = play_fg_k2[(play_fg_k2.x>=45) & (play_fg_k2.x<=75)]
kicker_result2018 = pd.DataFrame(play_fg_k2.groupby(by=['displayName','possessionTeam'])['good_kick','attempt'].sum()) 

kicker_result2018 = kicker_result2018.reset_index()
kick_accuracy = []
for i in range(len(kicker_result2018)):
    kick_accuracy.append(kicker_result2018['good_kick'][i]/kicker_result2018['attempt'][i])

kicker_result2018 = pd.concat([kicker_result2018,
                               pd.DataFrame(kick_accuracy, columns=['kick_accuracy'])] ,axis=1)

kicker_result2018.sort_values(by = ['kick_accuracy','good_kick'],ascending=False)

A guy from SEA made 5 of 7 and 0.71!! (amount of that long-distance goals is also important!)

Let find SEA Special Team field-goals in season 2018 on map(find kicks from rating table)

In [ ]:
# map for kickers
# home and away field-goal attempts
SEA_2018_K1 = pd.merge(tracking2018[(tracking2018.position=='K') & (tracking2018.event =='ball_snap')],
                    play[(play.possessionTeam=='SEA') & (play.specialTeamsPlayType=='Field Goal')],
                    how = 'inner', on = ["gameId","playId"])
SEA_2018_K1 = pd.merge(SEA_2018_K1, game, how = 'inner', on = "gameId")
SEA_2018_K1 = SEA_2018_K1[['x','y','playDirection','specialTeamsResult','homeTeamAbbr','visitorTeamAbbr']]
#
SEA_2018_K2 = pd.merge(tracking2018[(tracking2018.position=='K') & (tracking2018.event =='field_goal_attempt')],
                    play[(play.possessionTeam=='SEA') & (play.specialTeamsPlayType=='Field Goal')],
                    how = 'inner', on = ["gameId","playId"])
SEA_2018_K2 = pd.merge(SEA_2018_K2, game, how = 'inner', on = "gameId")
SEA_2018_K2 = SEA_2018_K2[['x','y','playDirection','specialTeamsResult','homeTeamAbbr','visitorTeamAbbr']]

for i in range(len(SEA_2018_K1)):
    if SEA_2018_K1['homeTeamAbbr'][i]=='SEA' and SEA_2018_K1['playDirection'][i]=='left':
        SEA_2018_K1['x'][i] = 120 - SEA_2018_K1['x'][i]
        SEA_2018_K1['y'][i] = 53.3 - SEA_2018_K1['y'][i]
    elif SEA_2018_K1['visitorTeamAbbr'][i]=='SEA' and SEA_2018_K1['playDirection'][i]=='right':
        SEA_2018_K1['x'][i] = 120 - SEA_2018_K1['x'][i]
        SEA_2018_K1['y'][i] = 53.3 - SEA_2018_K1['y'][i]
#
for i in range(len(SEA_2018_K2)):
    if SEA_2018_K2['homeTeamAbbr'][i]=='SEA' and SEA_2018_K2['playDirection'][i]=='left':
        SEA_2018_K2['x'][i] = 120 - SEA_2018_K2['x'][i]
        SEA_2018_K2['y'][i] = 53.3 - SEA_2018_K2['y'][i]
    elif SEA_2018_K2['visitorTeamAbbr'][i]=='SEA' and SEA_2018_K2['playDirection'][i]=='right':
        SEA_2018_K2['x'][i] = 120 - SEA_2018_K2['x'][i]
        SEA_2018_K2['y'][i] = 53.3 - SEA_2018_K2['y'][i]

imageFile2 = cbook.get_sample_data('/kaggle/input/nfl2022/nfl_field5.png')
image2 = plt.imread(imageFile2)
extent = -60, 60, 0, 53.3

plt.plot(SEA_2018_K1[['x']][SEA_2018_K1.specialTeamsResult=='Kick Attempt Good']*1215/120,
         SEA_2018_K1[['y']][SEA_2018_K1.specialTeamsResult=='Kick Attempt Good']*553/53.3, '+', markersize=12, color='blue', alpha=0.9)
plt.plot(SEA_2018_K2[['x']][SEA_2018_K2.specialTeamsResult=='Kick Attempt Good']*1215/120,
         SEA_2018_K2[['y']][SEA_2018_K2.specialTeamsResult=='Kick Attempt Good']*553/53.3, 'o', markersize=8, color='cyan', alpha=0.9)


plt.plot(SEA_2018_K1[['x']][SEA_2018_K1.specialTeamsResult=='Kick Attempt No Good']*1215/120,
         SEA_2018_K1[['y']][SEA_2018_K1.specialTeamsResult =='Kick Attempt No Good']*553/53.3, '+', markersize=12, color='orange', alpha=0.9)
plt.plot(SEA_2018_K2[['x']][SEA_2018_K2.specialTeamsResult=='Kick Attempt No Good']*1215/120,
         SEA_2018_K2[['y']][SEA_2018_K2.specialTeamsResult =='Kick Attempt No Good']*553/53.3, 'o', markersize=8, color='red', alpha=0.9)


plt.rcParams["figure.figsize"] = (12,5)
plt.axis('off')
plt.title('SEA field-goal attempts (2018)')
plt.imshow(image2)

We can see starting point of kicker (at moment of snap) marked by '+' and final point of kicker (at moment of attempt) marked by 'o'.
Long distance field-goals are blue(cyan) circles between 35 yards lines. 